In [4]:
%pip install torch torchvision torchaudio torch-geometric
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics.pairwise import cosine_similarity
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 708.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 1.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kern

In [8]:
df = pd.read_csv('/Users/hetjivani/NIAR/Neuro-Imaging-Analysis-and-Research/GroupE/Final_Code/clinical_modeling_dataset.csv')
df = df.dropna(subset=["sara"])

In [9]:
fc_features = [col for col in df.columns if col.startswith("f")]
X = df[fc_features].values.astype(np.float32)
y = df["sara"].values.astype(np.float32)

In [11]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
cos_sim = cosine_similarity(X)
np.fill_diagonal(cos_sim, 0)
edge_index = np.array(np.where(cos_sim > 0.9))
edge_index = torch.tensor(edge_index, dtype=torch.long)

In [13]:
X_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.float).unsqueeze(1)
data = Data(x=X_tensor, edge_index=edge_index, y=y_tensor)

In [14]:
train_idx, test_idx = train_test_split(np.arange(len(df)), test_size=0.2, random_state=42)
train_mask = torch.zeros(len(df), dtype=torch.bool)
test_mask = torch.zeros(len(df), dtype=torch.bool)
train_mask[train_idx] = True
test_mask[test_idx] = True
data.train_mask = train_mask
data.test_mask = test_mask

In [15]:
class DASTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(DASTGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

device = torch.device("cpu")  # Force CPU
model = DASTGCN(in_channels=X.shape[1], hidden_channels=64, out_channels=1).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

In [16]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out[data.test_mask].squeeze().cpu().numpy()
        trues = data.y[data.test_mask].squeeze().cpu().numpy()
        rmse = np.sqrt(mean_squared_error(trues, preds))
        r2 = r2_score(trues, preds)
        return rmse, r2

In [17]:
for epoch in range(200):
    loss = train()
    if epoch % 20 == 0:
        rmse, r2 = evaluate()
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f}")

Epoch 000 | Loss: 308.8852 | RMSE: 60.0368 | R²: -112.1331
Epoch 020 | Loss: 44.4024 | RMSE: 67.4330 | R²: -141.7248
Epoch 040 | Loss: 3.8398 | RMSE: 59.6266 | R²: -110.5924
Epoch 060 | Loss: 0.6259 | RMSE: 57.6961 | R²: -103.4833
Epoch 080 | Loss: 0.1021 | RMSE: 57.2179 | R²: -101.7585
Epoch 100 | Loss: 0.0160 | RMSE: 57.1938 | R²: -101.6721
Epoch 120 | Loss: 0.0011 | RMSE: 57.1774 | R²: -101.6130
Epoch 140 | Loss: 0.0001 | RMSE: 57.1710 | R²: -101.5902
Epoch 160 | Loss: 0.0000 | RMSE: 57.1720 | R²: -101.5937
Epoch 180 | Loss: 0.0000 | RMSE: 57.1721 | R²: -101.5942


In [18]:
final_rmse, final_r2 = evaluate()
print(f"\n✅ Final DAST-GCN Results → RMSE: {final_rmse:.4f}, R²: {final_r2:.4f}")


✅ Final DAST-GCN Results → RMSE: 57.1720, R²: -101.5939
